In [1]:
##Hotel interface

In [ ]:
import java.rmi.*;

public interface HotelInterface extends Remote {
    String bookRoom(String guestName) throws RemoteException;
    String cancelBooking(String guestName) throws RemoteException;
}

In [3]:
##Hotel Server

In [ ]:
import java.rmi.*;
import java.rmi.registry.LocateRegistry;
import java.rmi.server.UnicastRemoteObject;
import java.util.HashMap;

// Interface implementation is embedded here
public class HotelServer extends UnicastRemoteObject implements HotelInterface {
    private HashMap<String, String> bookings;

    public HotelServer() throws RemoteException {
        bookings = new HashMap<>();
    }

    public synchronized String bookRoom(String guestName) throws RemoteException {
        if (bookings.containsKey(guestName)) {
            return "Guest " + guestName + " already has a booking.";
        } else {
            bookings.put(guestName, "Booked");
            return "Room booked for " + guestName;
        }
    }

    public synchronized String cancelBooking(String guestName) throws RemoteException {
        if (bookings.containsKey(guestName)) {
            bookings.remove(guestName);
            return "Booking cancelled for " + guestName;
        } else {
            return "No booking found for " + guestName;
        }
    }

    public static void main(String[] args) {
        try {
            LocateRegistry.createRegistry(1099); // Start RMI Registry
            HotelServer hotel = new HotelServer(); // Create object
            Naming.rebind("HotelService", hotel); // Bind to registry
            System.out.println("Hotel Server is ready...");
        } catch (Exception e) {
            System.out.println("Server Exception: " + e.getMessage());
            e.printStackTrace();
        }
    }
}

In [5]:
##Hotel client

In [ ]:
import java.rmi.Naming;
import java.util.Scanner;

public class HotelClient {
    public static void main(String[] args) {
        try {
            HotelInterface hotel = (HotelInterface) Naming.lookup("rmi://localhost/HotelService");
            Scanner scanner = new Scanner(System.in);
            int choice;

            do {
                System.out.println("\n=== Hotel Booking System ===");
                System.out.println("1. Book a Room");
                System.out.println("2. Cancel Booking");
                System.out.println("3. Exit");
                System.out.print("Enter your choice: ");
                choice = scanner.nextInt();
                scanner.nextLine(); // consume newline

                switch (choice) {
                    case 1:
                        System.out.print("Enter guest name to book: ");
                        String nameToBook = scanner.nextLine();
                        System.out.println(hotel.bookRoom(nameToBook));
                        break;

                    case 2:
                        System.out.print("Enter guest name to cancel: ");
                        String nameToCancel = scanner.nextLine();
                        System.out.println(hotel.cancelBooking(nameToCancel));
                        break;

                    case 3:
                        System.out.println("Exiting... Goodbye!");
                        break;

                    default:
                        System.out.println("Invalid choice.");
                }

            } while (choice != 3);

            scanner.close();

        } catch (Exception e) {
            System.out.println("Client Exception: " + e.getMessage());
            e.printStackTrace();
        }
    }
}